# CS 224N Final Project - Evaluating Fine-Tuned Models on WinoDict Dataset
By: Christopher Pondoc, Joseph Guman, and Joseph O'Brien

In [1]:
import torch
print("Using GPU: " + str(torch.cuda.is_available()))

Using GPU: True


## Load in GPT-2 Model
Using HuggingFace Transformers

In [6]:
import torch
from transformers import AutoTokenizer, GPT2LMHeadModel
tokenizer = AutoTokenizer.from_pretrained("weights/GWithFineTunedSentencesG2GInitializationT")
model = GPT2LMHeadModel.from_pretrained("weights/GWithFineTunedSentencesG2GInitialization")
model.to('cuda')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout)

## Load in WinoDict Dataset and Real-to-Fake Words
HuggingFace + Previously Generated.

In [7]:
import pandas as pd
first_set = pd.read_csv("winodict/prob1_of_5.csv")
rtf_df = pd.read_csv('datasets/realtofake.csv', header=None, index_col=0, squeeze = True)
rtf = rtf_df.to_dict()

/tmp/ipykernel_16474/341005566.py:3: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  rtf_df = pd.read_csv('datasets/realtofake.csv', header=None, index_col=0, squeeze = True)


## Evaluating on One Example
Writing a function that is reusable and works for one example

In [8]:
def evaluate_winodict(example):
    # First, replace the word with each of the options
    if ('_' in example['sentence']):
        # Change 'the' to lowercase
        first_choice, second_choice = example['option1'], example['option2']
        if (first_choice[:4] == "The "):
            first_choice = "the " + first_choice[4:]
        if (second_choice[:4] == "The "):
            second_choice = "the " + second_choice[4:]
    
        # Get the fake word and replace the definition
        fake_word = rtf[example['lemma']]
        example['definition'] = example['definition'].replace(example['fake_lemma'], fake_word)
        example['sentence'] = example['sentence'].replace(example['fake_lemma'], fake_word)
        
        # Replace the text
        first_text, second_text = example['sentence'], example['sentence']
        pronoun_loc = example['sentence'].index('_')
        first_option = example['definition'] + " " + first_text[:pronoun_loc] + first_choice + first_text[pronoun_loc + 1:]
        second_option = example['definition'] + " " + second_text[:pronoun_loc] + second_choice + second_text[pronoun_loc + 1:]
    
        # Tokenize each string and produce labels
        first_inputs, second_inputs = tokenizer(first_option, return_tensors="pt").to('cuda'), tokenizer(second_option, return_tensors="pt").to('cuda')

        # Create the first token labels
        first_masked_tokens = tokenizer(example['definition'] + " " + first_text[:pronoun_loc] + first_choice, return_tensors="pt")
        first_labels = first_masked_tokens["input_ids"][0]
        first_mask = torch.full((1, first_labels.shape[0]), -100)
        first_fill = tokenizer(first_text[pronoun_loc + 1:], return_tensors="pt")["input_ids"]
        final_first_labels = torch.cat((first_mask, first_fill), dim=1).to('cuda')

        # Create the second token labels
        second_masked_tokens = tokenizer(example['definition'] + " " + second_text[:pronoun_loc] + second_choice, return_tensors="pt")
        second_labels = second_masked_tokens["input_ids"][0]
        second_mask = torch.full((1, second_labels.shape[0]), -100)
        second_fill = tokenizer(second_text[pronoun_loc + 1:], return_tensors="pt")["input_ids"]
        final_second_labels = torch.cat((second_mask, second_fill), dim=1).to('cuda')
        
        # Evaluate the model on each example and check
        first_loss = model(**first_inputs, labels=final_first_labels).loss
        second_loss = model(**second_inputs, labels=final_second_labels).loss
        
        # Write down the correct value and check
        if (first_loss < second_loss):
            return (int(example['label']) == 0)
        else:
            return (int(example['label']) == 1)
        
    # First, replace the word with each of the options
    '''if ('_' in example['sentence']):
        # Change 'the' to lowercase
        first_choice, second_choice = example['option1'], example['option2']
        if (first_choice[:4] == "The "):
            first_choice = "the " + first_choice[4:]
        if (second_choice[:4] == "The "):
            second_choice = "the " + second_choice[4:]

        # Replace the text
        first_text, second_text = example['sentence'], example['sentence']
        pronoun_loc = example['sentence'].index('_')
        first_option = example['definition'] + " " + first_text[:pronoun_loc] + first_choice + first_text[pronoun_loc + 1:]
        second_option = example['definition'] + " " + second_text[:pronoun_loc] + second_choice + second_text[pronoun_loc + 1:]

        # Tokenize each string and produce labels
        first_inputs, second_inputs = tokenizer(first_option, return_tensors="pt"), tokenizer(second_option, return_tensors="pt")

        # Create the first token labels
        first_masked_tokens = tokenizer(example['definition'] + " " + first_text[:pronoun_loc] + first_choice, return_tensors="pt")
        first_labels = first_masked_tokens["input_ids"][0]
        first_mask = torch.full((1, first_labels.shape[0]), -100)
        first_fill = tokenizer(first_text[pronoun_loc + 1:], return_tensors="pt")["input_ids"]
        final_first_labels = torch.cat((first_mask, first_fill), dim=1)

        # Create the second token labels
        second_masked_tokens = tokenizer(example['definition'] + " " + second_text[:pronoun_loc] + second_choice, return_tensors="pt")
        second_labels = second_masked_tokens["input_ids"][0]
        second_mask = torch.full((1, second_labels.shape[0]), -100)
        second_fill = tokenizer(second_text[pronoun_loc + 1:], return_tensors="pt")["input_ids"]
        final_second_labels = torch.cat((second_mask, second_fill), dim=1)

        # Evaluate the model on each example and check
        first_loss = model(**first_inputs, labels=final_first_labels).loss
        second_loss = model(**second_inputs, labels=final_second_labels).loss
        
        # Write down the correct value and check
        if (first_loss < second_loss):
            return (int(example['label']) == 0)
        else:
            return (int(example['label']) == 1)'''

## Evaluating WinoDict on GPT-2
Looking specifically at `WinoDict`, with the first generated examples and adding in the definition and substituting in the word.

In [9]:
correct, total = 0, 0
for index, row in first_set.iterrows():
    if (row['lemma'] != "lemma"):
        total += 1
        correct += int(evaluate_winodict(row))
        #correct += evaluate_winodict(row)
        print(correct)
    
print("GPT-2 Medium achieved a score of: " + str(float(correct) / float(total)))

0
0
1
1
2
2
3
3
3
4
5
5
6
6
7
8
8
9
9
10
10
11
11
12
13
13
14
14
15
15
15
16
16
17
18
18
18
19
20
21
22
22
22
23
23
24
25
25
25
26
27
27
27
28
28
29
29
30
31
31
31
32
33
33
33
34
34
35
35
36
36
37
38
39
39
40
40
41
41
42
42
43
44
44
45
45
46
46
47
47
47
48
49
49
49
49
49
50
51
51
51
51
51
52
53
53
53
54
54
55
55
56
57
57
57
58
59
59
59
59
59
60
60
61
61
62
63
64
64
65
65
66
67
67
68
68
68
69
70
70
70
71
71
72
73
73
74
74
74
75
76
76
77
77
77
78
79
79
80
80
80
81
81
82
83
83
83
84
85
85
85
86
87
87
87
88
88
89
89
90
91
91
91
92
92
93
93
94
95
95
96
97
98
98
99
100
100
101
101
102
103
103
104
104
105
106
107
107
107
108
109
109
109
110
110
110
111
112
112
112
113
113
114
114
114
115
115
115
116
116
116
117
118
118
119
119
119
120
121
121
121
122
123
123
123
124
124
125
126
126
127
128
128
129
129
130
130
131
132
132
133
133
133
134
134
135
135
136
137
137
137
137
137
138
139
139
140
141
141
142
142
143
143
144
145
145
145
146
146
146
146
147
148
149
150
151
151
152
152
152
153
153
153
15